<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: Validating Shifted Kerr-Schild initial data against ETK version: 

## Author: Patrick Nelson

**Notebook Status:** <font color='green'><b>Validated</b></font>

**Validation Notes:** This module validates all expressions used to set up initial data in 
* [Tutorial-ADM_Initial_Data-ShiftedKerrSchild](../Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb)

against the C-code implementation of these expressions found in the original (trusted) [`GiRaFFEfood` Einstein Toolkit thorn](link), and confirms roundoff-level agreement.

### NRPy+ Source Code for this module: 
* [BSSN/ShiftedKerrSchild.py](../../edit/BSSN/ShiftedKerrSchild.py) [\[**tutorial**\]](../Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb) Generates Exact Wald initial data

## Introduction:

This notebook validates the initial data routines that set up the Shifted Kerr-Schild initial spacetime data against the ETK implementation of the same equations.

When this notebook is run, the significant digits of agreement between the old ETK and new NRPy+ versions of the algorithm will be evaluated. If the agreement falls below a thresold, the point, quantity, and level of agreement are reported [here](#compile_run).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#setup): Set up core functions and parameters for unit testing the initial data algorithms
    1. [Step 1.a](#spacetime) Generate the spacetime metric
    1. [Step 1.b](#download) Download original ETK files
    1. [Step 1.c](#free_params) Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
    1. [Step 1.d](#interface) Create dummy files for the CCTK version of the code
1. [Step 2](#mainc): `ShiftedKerrSchild_unit_test.c`: The Main C Code
    1. [Step 2.a](#compile_run): Compile and run the code to validate the output
1. [Step 3](#drift_notes): Output this notebook to $\LaTeX$-formatted PDF file
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for unit testing the initial data algorithms" \[Back to [top](#toc)\]

$$\label{setup}$$

We'll start by appending the relevant paths to `sys.path` so that we can access sympy modules in other places. Then, we'll import NRPy+ core functionality and set up a directory in which to carry out our test. We will also declare the gridfunctions that are needed for this portion of the code.

In [1]:
import os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import outCfunction, lhrh # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir = "Validation/"
cmd.mkdir(out_dir)

thismodule = "Start_to_Finish_UnitTest-GiRaFFEfood_NRPy"

# Register the gridfunctions we need for this function
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01")
betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU")
alpha = gri.register_gridfunctions("AUXEVOL","alpha")

<a id='spacetime'></a>

## Step 1.a: Generate the spacetime metric \[Back to [top](#toc)\]
$$\label{spacetime}$$

While many of the initial data we will use assume a flat background spacetime, some will require a specific metric. We will set those up as needed here.

In [2]:
# Exact Wald is more complicated. We'll need the Shifted Kerr Schild metric in Cartesian coordinates.
import BSSN.ShiftedKerrSchild as sks
sks.ShiftedKerrSchild(True)
import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()
# Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv()
gammaDD = ixp.zerorank2()
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                gammaDD[i][j] += drrefmetric__dx_0UDmatrix[(k,i)]*drrefmetric__dx_0UDmatrix[(l,j)]*sks.gammaSphDD[k][l].subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])
betaU = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        betaU[i] += dx__drrefmetric_0UDmatrix[(i,j)]*sks.betaSphU[j].subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])

# We only need to set alpha and betaU in C for the original Exact Wald
name = "Shifted_Kerr_Schild_initial_metric"
desc = "Generate a spinning black hole with Shifted Kerr Schild metric."
values_to_print = [
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD00"),rhs=gammaDD[0][0]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD01"),rhs=gammaDD[0][1]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD02"),rhs=gammaDD[0][2]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD11"),rhs=gammaDD[1][1]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD12"),rhs=gammaDD[1][2]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD22"),rhs=gammaDD[2][2]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","betaU0"),rhs=betaU[0]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","betaU1"),rhs=betaU[1]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","betaU2"),rhs=betaU[2]),
                   lhrh(lhs=gri.gfaccess("auxevol_gfs","alpha"),rhs=sks.alphaSph.subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1]))
                  ]

outCfunction(
    outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *params,REAL *xx[3],REAL *auxevol_gfs",
    body     = fin.FD_outputC("returnstring",values_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts ="AllPoints,Read_xxs")


Output C function Shifted_Kerr_Schild_initial_metric() to file Validation/Shifted_Kerr_Schild_initial_metric.h


<a id='download'></a>

## Step 1.b: Download original ETK files \[Back to [top](#toc)\]

$$\label{download}$$

Here, we download the relevant portion of the original `GiRaFFE` code from Bitbucket. 

In [3]:
# First download the original GiRaFFE source code
import urllib

original_file_url  = [
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0a82c822748baf754c153db484d8bd2d0b7e39cb/ShiftedKerrSchild/src/ShiftedKerrSchild.c",
                     ]
original_file_name = [
                      "ShiftedKerrSchild.c",
                     ]

for i in range(len(original_file_url)):
    original_file_path = os.path.join(out_dir,original_file_name[i])

    # Then download the original GiRaFFE source code
    # We try it here in a couple of ways in an attempt to keep
    # the code more portable
    try:
        original_file_code = urllib.request.urlopen(original_file_url[i]).read().decode('utf-8')
    except:
        original_file_code = urllib.urlopen(original_file_url[i]).read().decode('utf-8')

    # Write down the file the original GiRaFFE source code
    with open(original_file_path,"w") as file:
        file.write(original_file_code)


<a id='free_params'></a>

## Step 1.c: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]

$$\label{free_params}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets some basic grid parameters as well as the speed limit parameter we need for this function.

In [4]:
# Step 3.d
# Step 3.d.ii: Set free_parameters.h
with open(os.path.join(out_dir,"free_parameters.h"),"w") as file:
    file.write("""
// Set free-parameter values.

const int NGHOSTS = 3;

// Set free-parameter values for the initial data.
// Override parameter defaults with values based on command line arguments and NGHOSTS.
const int Nx0x1x2 = 5;
params.Nxx0 = Nx0x1x2;
params.Nxx1 = Nx0x1x2;
params.Nxx2 = Nx0x1x2;
params.Nxx_plus_2NGHOSTS0 = params.Nxx0 + 2*NGHOSTS;
params.Nxx_plus_2NGHOSTS1 = params.Nxx1 + 2*NGHOSTS;
params.Nxx_plus_2NGHOSTS2 = params.Nxx2 + 2*NGHOSTS;
// Step 0d: Set up space and time coordinates
// Step 0d.i: Declare \Delta x^i=dxx{0,1,2} and invdxx{0,1,2}, as well as xxmin[3] and xxmax[3]:
const REAL xxmin[3] = {-1.5,-1.5,-1.5};
const REAL xxmax[3] = { 1.5, 1.5, 1.5};

params.dxx0 = (xxmax[0] - xxmin[0]) / ((REAL)params.Nxx0);
params.dxx1 = (xxmax[1] - xxmin[1]) / ((REAL)params.Nxx1);
params.dxx2 = (xxmax[2] - xxmin[2]) / ((REAL)params.Nxx2);
params.invdx0 = 1.0 / params.dxx0;
params.invdx1 = 1.0 / params.dxx1;
params.invdx2 = 1.0 / params.dxx2;
params.r0 = 0.4;
params.a = 0.0;
\n""")

# Generates declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(out_dir))

<a id='interface'></a>

## Step 1.d: Create dummy files for the CCTK version of the code \[Back to [top](#toc)\]

$$\label{interface}$$

The original `GiRaFFE` code depends on some functionalities of the CCTK. Since we only care about this one small function, we can get around this by creating some nearly-empty, non-functional files that can be included to satisfy the pre-processor without changing functionality. We will later replace what little functionality we need with some basic global variables and macros.

In [5]:
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
with open(os.path.join(out_dir,"cctk.h"),"w") as file:
    file.write("""//""")

with open(os.path.join(out_dir,"cctk_Arguments.h"),"w") as file:
    file.write("""#define DECLARE_CCTK_ARGUMENTS //
#define CCTK_ARGUMENTS void
""")

with open(os.path.join(out_dir,"cctk_Parameters.h"),"w") as file:
    file.write("""#define DECLARE_CCTK_PARAMETERS //
""")

<a id='mainc'></a>

# Step 2: `ShiftedKerrSchild_unit_test.C`: The Main C Code \[Back to [top](#toc)\]

$$\label{mainc}$$

Now that we have our vector potential and analytic magnetic field to compare against, we will start writing our unit test. We'll also import common C functionality, define `REAL`, the number of ghost zones, and the faces, and set the standard macros for NRPy+ style memory access.

In [6]:
%%writefile $out_dir/ShiftedKerrSchild_unit_test.C
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "stdint.h" // Needed for Windows GCC 6.x compatibility

#define REAL double
#include "declare_Cparameters_struct.h"

// Standard NRPy+ memory access:
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )

// Standard formula to calculate significant digits of agreement:
#define SDA(a,b) 1.0-log10(2.0*fabs(a-b)/(fabs(a)+fabs(b)))

// Memory access definitions for NRPy+
#define GAMMADD00GF 0
#define GAMMADD01GF 1
#define GAMMADD02GF 2
#define GAMMADD11GF 3
#define GAMMADD12GF 4
#define GAMMADD22GF 5
#define BETAU0GF 6
#define BETAU1GF 7
#define BETAU2GF 8
#define ALPHAGF 9
#define KDD00GF 10
#define KDD01GF 11
#define KDD02GF 12
#define KDD11GF 13
#define KDD12GF 14
#define KDD22GF 15
#define NUM_AUXEVOL_GFS 16

// Include the functions that we want to test:
#include "Shifted_Kerr_Schild_initial_metric.h"

// Define CCTK macros
#define CCTK_REAL double
#define CCTK_INT int
#define CCTK_VPARAMWARN(...) //
#define CCTK_EQUALS(a,b) 1
struct cGH{};
const cGH* cctkGH;

// More definitions to interface with ETK code:
const int cctk_lsh[3] = {11,11,11};
const int grid_size = cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2];

Overwriting Validation//ShiftedKerrSchild_unit_test.C


In [7]:
import os
gfs_list = ['x','y','z','r','SKSgrr','SKSgrth','SKSgrph','SKSgthth','SKSgthph','SKSgphph','SKSbetar','SKSbetath','SKSbetaph']
with open(os.path.join(out_dir,"ShiftedKerrSchild_unit_test.C"), 'a') as file:
    for gf in gfs_list:
        file.write("CCTK_REAL *"+gf+" = (CCTK_REAL *)malloc(sizeof(CCTK_REAL)*grid_size);\n")

In [8]:
%%writefile -a $out_dir/ShiftedKerrSchild_unit_test.C
CCTK_REAL *alp;
CCTK_REAL *betax;
CCTK_REAL *betay;
CCTK_REAL *betaz;
CCTK_REAL *gxx;
CCTK_REAL *gxy;
CCTK_REAL *gxz;
CCTK_REAL *gyy;
CCTK_REAL *gyz;
CCTK_REAL *gzz;
CCTK_REAL *kxx;
CCTK_REAL *kxy;
CCTK_REAL *kxz;
CCTK_REAL *kyy;
CCTK_REAL *kyz;
CCTK_REAL *kzz;

CCTK_REAL KerrSchild_radial_shift;
CCTK_REAL BH_mass;
CCTK_REAL BH_spin;


// Dummy ETK function:
#define CCTK_GFINDEX3D(cctkGH,i,j,k) (i) + cctk_lsh[0] * ( (j) + cctk_lsh[1] * (k) )

#include "ShiftedKerrSchild.c"

int main() {
    paramstruct params;
#include "set_Cparameters_default.h"

    // Step 0c: Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"
#include "set_Cparameters-nopointer.h"

    // Set CCTK parameters to match NRPy+ parameters
    KerrSchild_radial_shift = r0;
    BH_mass = M;
    BH_spin = a;

    // Step 0d.ii: Set up uniform coordinate grids
    REAL *xx[3];
    xx[0] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS0);
    xx[1] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS1);
    xx[2] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS2);
    for(int j=0;j<Nxx_plus_2NGHOSTS0;j++) xx[0][j] = xxmin[0] + (j-NGHOSTS)*dxx0;
    for(int j=0;j<Nxx_plus_2NGHOSTS1;j++) xx[1][j] = xxmin[1] + (j-NGHOSTS)*dxx1;
    for(int j=0;j<Nxx_plus_2NGHOSTS2;j++) xx[2][j] = xxmin[2] + (j-NGHOSTS)*dxx2;

    for(int k=0;k<Nxx_plus_2NGHOSTS2;k++)
        for(int j=0;j<Nxx_plus_2NGHOSTS1;j++)
            for(int i=0;i<Nxx_plus_2NGHOSTS0;i++) {
                int index = CCTK_GFINDEX3D(cctkGH,i,j,k);
                x[index] = xx[0][i];
                y[index] = xx[1][j];
                z[index] = xx[2][k];
                r[index] = sqrt(x[index]*x[index] + y[index]*y[index] + z[index]*z[index]);
    }

    //for(int j=0;j<Nxx_plus_2NGHOSTS0;j++) printf("x[%d] = %.5e\n",j,xx[0][j]);

    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS2 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS0);
    REAL *auxevol_ETK_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS2 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS0);

    // Memory access for metric gridfunctions for Exact Wald:
    gxx = auxevol_ETK_gfs + (grid_size*GAMMADD00GF);
    gxy = auxevol_ETK_gfs + (grid_size*GAMMADD01GF);
    gxz = auxevol_ETK_gfs + (grid_size*GAMMADD02GF);
    gyy = auxevol_ETK_gfs + (grid_size*GAMMADD11GF);
    gyz = auxevol_ETK_gfs + (grid_size*GAMMADD12GF);
    gzz = auxevol_ETK_gfs + (grid_size*GAMMADD22GF);
    alp = auxevol_ETK_gfs + (grid_size*ALPHAGF);
    betax = auxevol_ETK_gfs + (grid_size*BETAU0GF);
    betay = auxevol_ETK_gfs + (grid_size*BETAU1GF);
    betaz = auxevol_ETK_gfs + (grid_size*BETAU2GF);
    kxx = auxevol_ETK_gfs + (grid_size*KDD00GF);
    kxy = auxevol_ETK_gfs + (grid_size*KDD01GF);
    kxz = auxevol_ETK_gfs + (grid_size*KDD02GF);
    kyy = auxevol_ETK_gfs + (grid_size*KDD11GF);
    kyz = auxevol_ETK_gfs + (grid_size*KDD12GF);
    kzz = auxevol_ETK_gfs + (grid_size*KDD22GF);

    Shifted_Kerr_Schild_initial_metric(&params,xx,auxevol_gfs);
    ShiftedKS_ID();

    int all_agree = 1;

    for(int i0=0;i0<Nxx_plus_2NGHOSTS0;i0++){
        for(int i1=0;i1<Nxx_plus_2NGHOSTS1;i1++){
            for(int i2=0;i2<Nxx_plus_2NGHOSTS2;i2++){
                if(SDA(auxevol_gfs[IDX4S(BETAU0GF, i0,i1,i2)],betax[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)])<10.0){
                    printf("Quantity betaU0 only agrees with the original GiRaFFE to %.2f digits at i0,i1,i2=%d,%d,%d!\n",
                           SDA(auxevol_gfs[IDX4S(BETAU0GF, i0,i1,i2)],betax[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]),i0,i1,i2);
                    all_agree=0;
                }
                if(SDA(auxevol_gfs[IDX4S(BETAU1GF, i0,i1,i2)],betay[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)])<10.0){
                    printf("Quantity betaU1 only agrees with the original GiRaFFE to %.2f digits at i0,i1,i2=%d,%d,%d!\n",
                           SDA(auxevol_gfs[IDX4S(BETAU1GF, i0,i1,i2)],betay[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]),i0,i1,i2);
                    all_agree=0;
                }
                if(SDA(auxevol_gfs[IDX4S(BETAU2GF, i0,i1,i2)],betaz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)])<10.0){
                    printf("Quantity betaU2 only agrees with the original GiRaFFE to %.2f digits at i0,i1,i2=%d,%d,%d!\n",
                           SDA(auxevol_gfs[IDX4S(BETAU2GF, i0,i1,i2)],betaz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]),i0,i1,i2);
                    all_agree=0;
                }
                if(SDA(auxevol_gfs[IDX4S(GAMMADD00GF, i0,i1,i2)],gxx[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)])<10.0){
                    printf("Quantity betaU0 only agrees with the original GiRaFFE to %.2f digits at i0,i1,i2=%d,%d,%d!\n",
                           SDA(auxevol_gfs[IDX4S(GAMMADD00GF, i0,i1,i2)],gxx[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]),i0,i1,i2);
                    all_agree=0;
                }
                if(SDA(auxevol_gfs[IDX4S(GAMMADD11GF, i0,i1,i2)],gyy[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)])<10.0){
                    printf("Quantity betaU1 only agrees with the original GiRaFFE to %.2f digits at i0,i1,i2=%d,%d,%d!\n",
                           SDA(auxevol_gfs[IDX4S(GAMMADD11GF, i0,i1,i2)],gyy[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]),i0,i1,i2);
                    all_agree=0;
                }
                if(SDA(auxevol_gfs[IDX4S(GAMMADD22GF, i0,i1,i2)],gzz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)])<10.0){
                    printf("Quantity betaU2 only agrees with the original GiRaFFE to %.2f digits at i0,i1,i2=%d,%d,%d!\n",
                           SDA(auxevol_gfs[IDX4S(GAMMADD22GF, i0,i1,i2)],gzz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]),i0,i1,i2);
                    all_agree=0;
                }
                //printf("NRPy: %.15e,%.15e,%.15e\n",auxevol_gfs[IDX4S(BETAU0GF, i0,i1,i2)],auxevol_gfs[IDX4S(BETAU1GF, i0,i1,i2)],auxevol_gfs[IDX4S(BETAU2GF, i0,i1,i2)]);
                //printf("CCTK: %.15e,%.15e,%.15e\n",betax[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)],betay[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)],betaz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]);
                //printf("NRPy: %.15e,%.15e,%.15e\n",auxevol_gfs[IDX4S(GAMMADD01GF, i0,i1,i2)],auxevol_gfs[IDX4S(GAMMADD02GF, i0,i1,i2)],auxevol_gfs[IDX4S(GAMMADD12GF, i0,i1,i2)]);
                //printf("CCTK: %.15e,%.15e,%.15e\n",gxy[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)],gxz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)],gyz[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)]);
            }
        }
    }

    if(all_agree) printf("All quantities agree at all points!\n");

Appending to Validation//ShiftedKerrSchild_unit_test.C


In [9]:
with open(os.path.join(out_dir,"ShiftedKerrSchild_unit_test.C"), 'a') as file:
    for gf in gfs_list:
        file.write("    free("+gf+");\n")

In [10]:
%%writefile -a $out_dir/ShiftedKerrSchild_unit_test.C
    free(auxevol_gfs);
    free(auxevol_ETK_gfs);
}

Appending to Validation//ShiftedKerrSchild_unit_test.C


<a id='compile_run'></a>

## Step 2.a: Compile and run the code to validate the output \[Back to [top](#toc)\]

$$\label{compile_run}$$

Finally, we can compile and run the code we have written. Once run, this code will output the level of agreement between the two codes and some information to help interpret those numbers.

In [11]:
import time

print("Now compiling, should take ~2 seconds...\n")
start = time.time()
# cmd.C_compile(os.path.join(out_dir,"ShiftedKerrSchild_unit_test.c"), os.path.join(out_dir,"ShiftedKerrSchild_unit_test"))
!g++ -Ofast -fopenmp -march=native -funroll-loops Validation/ShiftedKerrSchild_unit_test.C -o Validation/ShiftedKerrSchild_unit_test -lstdc++
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

results_file = "out_ShiftedKerrSchild_test.txt"

# os.chdir(out_dir)
os.chdir(out_dir)
# cmd.Execute(os.path.join("GiRaFFEfood_NRPy_unit_test"))
cmd.Execute("ShiftedKerrSchild_unit_test",file_to_redirect_stdout=results_file)
os.chdir(os.path.join("../"))

Now compiling, should take ~2 seconds...

Finished in 0.3594996929168701 seconds.


(EXEC): Executing `taskset -c 0,1,2,3,4,5,6,7 ./ShiftedKerrSchild_unit_test `...
(BENCH): Finished executing in 0.2101430892944336 seconds.


Here, we add some emergency brakes so that if the output from the test isn't good, we throw an error to stop the notebook dead in its tracks. This way, our automatic testing infrastructure can let us know if something goes wrong. We will also print the output from the test for convenience's sake.

In [12]:
with open(os.path.join(out_dir,results_file),"r") as file:
    output = file.readline()
    print(output)
    if output!="All quantities agree at all points!\n": # If this isn't the first line of this file, something went wrong!
        sys.exit(1)

All quantities agree at all points!



<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.pdf](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [13]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy")

[NbConvertApp] WARNING | pattern 'Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb' matched no files
Created Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.tex, and
    compiled LaTeX file to PDF file Tutorial-Start_to_Finish_UnitTest-
    GiRaFFEfood_NRPy.pdf
